Choose Your Own Adventure! Your options, from simplest to the hardest:

* You are happy with what you've learned about the FFT and want to move on: ignore this notebook
* You want to understand how FFT works a little deeper: watch the YouTube video at https://youtu.be/h7apO7q16V0 , where in 30 minutes, the narrator goes step-by-step with beautiful visuals. Grasping why FFT is so efficient is a great achievement :)
* You want to fully work out an example: go through this notebook with $N=8$.
    * Won't do it in-class. Even with small $N$, it's tricky to follow how each step fits into the next. Actually implementing it in code is the easy part, usually a line or two.
    * But even if you perfectly understand the YouTube video, there is something to be said about doing it for yourself. One doesn't become a scientist only by watching videos, being able to do stuff is really what earns you a degree :)

## Example with $N=8$ 

We have $M=\log_2(8) = 3$, as well as $k=0, 1, \dots 7$ at every step. We also have
$$\omega = \exp\left(-\frac{2i\pi}8\right) \Rightarrow \omega^{n+8} = \omega^n,$$
which we will use a lot.

### Writing the pairings

The FFT algorithm actually starts from the real values and works its way up to the coefficients. Here, I will start from the coefficients and make my way down to the values, to better understand the notations.

#### First bisection

Second coefficient of the DFT is
\begin{align*}
    c_1 = E_1^{(0, 0)} & = \sum_{n=0}^3y_n e^{-2i\pi n/8} = \sum_{n=0}^3y_n \omega^{n} \\
    & = y_0 + \omega y_1 + \omega^2 y_2 + \omega^3 y_3 + \omega^4 y_4 + \omega^5 y_5 + \omega^6 y_6 + \omega^7 y_7 \\
    & = \underbrace{y_0 + \omega^2 y_2 + \omega^4 y_4 + \omega^6 y_6}_{E_1^{(1, 0)}} 
        + \omega\underbrace{\left(y_1 + \omega^2 y_3 + \omega^4 y_5 + \omega^6 y_7\right)}_{E_1^{(1, 1)}}   
\end{align*}

* Split and multiplication count as one operation for us.
* Very hard to actually figure out the exact number of ops quickly. Let's call this "unit operation", one that also happens in DFT for comparison: 
    **Bisection + multiplication = 1 operation**
* Verify for yourself that eqn.
$$E_k^{(m,j)} = \sum_{p=0}^{N/2^m - 1}y_{2^m p + j}\exp\left(-i\frac{2\pi k p}{N/2^m}\right), \quad j \in \{0\dots 2^m-1\}$$
    works here.

#### Second bisection

Let's do a similar split for $E_1^{(1, 0)}$, for $j=0, 1$:
\begin{align*}
    E_1^{(1, 0)} & = y_{0} + \omega^2 y_{2} + \omega^4 y_{4} + \omega^6 y_{6} \\
                 & = \underbrace{y_{0} + \omega^4 y_{4}}_{E_1^{(2, 0)}} 
        + \omega^2\underbrace{\left(y_{2} +  \omega^4 y_{6}\right)}_{E_1^{(2, 2)}}   
\end{align*}
Note how I "skipped" $E_1^{(2, 1)}$: that's because we reserve it for the split of $E_1^{(1, 1)}$.
\begin{align*}
    E_1^{(1, 1)} & = \underbrace{y_{1} + \omega^4 y_{5}}_{E_1^{(2, 1)}} 
        + \omega^2\underbrace{\left(y_{3} +  \omega^4 y_{7}\right)}_{E_1^{(2, 3)}}   
\end{align*}
**Two bisections + multiplication = 2 operations**

Or, in a shorter fashion,
$$E_1^{(1, j)}  = \underbrace{y_{j} + \omega^4 y_{j+4}}_{E_1^{(2, j)}} + \omega^2\underbrace{\left(y_{j+2} +  \omega^4 y_{j+6}\right)}_{E_1^{(2, j+2)}}.$$
I could plug index $m$ to make it more systematic, which would give me the general formulas of the previous sub-section.

#### Third (and final) bisection
Finally, we can split previous $E$'s in the last step:
\begin{align*}
    E_1^{(2, 0)} & = y_0 + \omega^4 y_{4} = E^{(3, 0)} + \omega^4 E^{(3, 4)}\\
    E_1^{(2, 1)} & = y_1 + \omega^4 y_{5} = E^{(3, 1)} + \omega^4 E^{(3, 5)}\\
    E_1^{(2, 2)} & = y_2 + \omega^4 y_{6} = E^{(3, 2)} + \omega^4 E^{(3, 6)}\\
    E_1^{(2, 3)} & = y_3 + \omega^4 y_{7} = E^{(3, 3)} + \omega^4 E^{(3, 7)},
\end{align*}

* **Four bisections + multiplication = 4 operations**
* More simply$E_1^{(3, j)} = y_j$.
* We can use these 8 values of all $k$: actually connected to the $\omega^{n+8}= \omega^n$ property.

At this point: 
* $1+2+4 = 7$ operations, for 8 coefficients. $8\times7 = 56$ operations.
* Recall: I promised $N\log_2N = 8\times3 = 24$ ops. What is going on?
* Recall DFT: $\sim64$ operations (a little less actually, but not much).
* Answer: we do not have to compute all $E_k^{(m, j)}$ for all $k$'s, because $\omega^{n+8}=\omega^n$. Kind of like I did for the last bisection: I was going to re-use the $E^{3, j}$ for all $k$'s, but something similar happens at all stages.
* It is easier to see by actually doing it.

### Calculating the Fourier coefficients

The FFT algorithm works backward from what I just showed.
Which makes sense: we know the $y_j$'s, not the $\gamma_k$'s.

#### First step: start from the values
* $m=M=3$,
* $j=0, 1,\dots 7$,
* $\forall k, \quad E_k^{(3, j)} = E^{(3, j)} = y_j$ , 
* **Number of ops:** $(N/2^m = 1) \times (2^m = 8) = 8 = N$.

Note that these ar far less flop-consuming operations than the next steps, and they are common to both DFT and FFT. Therefore, we will not count these operations for simplicity.

#### Second step
* $m=2, j=0, 1, 2, 3$
* Compute the "partial" DFTs:
    \begin{align*}
        & E_0^{(2, j)} = E^{(3, j)} + \omega^{0\times4}E^{(3, j+4)}= E^{(3, j)} + E^{(3, j+4)},\\
        & E_1^{(2, j)} = E^{(3, j)} + \omega^{1\times4}E^{(3, j+4)} = E^{(3, j)} + \omega^4 E^{(3, j+4)},
    \end{align*}
    but we can actually stop after these $4\times 2 = 8$ operations.
    This is where we use the property that $\omega^{n+8}=\omega^n$: 
    \begin{align*}
        & E_2^{(2, j)} = E^{(3, j)} + \omega^{2\times4}E^{(3, j+4)} = E^{(3, j)} + \underbrace{\omega^8}_{=1} E^{(3, j+4)} = E_0^{(2, j)},\\
        & E_3^{(2, j)} = E^{(3, j)} + \omega^{3\times4}E^{(3, j+4)} = E^{(3, j)} + \underbrace{\omega^{12}}_{=\omega^4} E^{(3, j+4)} = E_1^{(2, j)},
    \end{align*}
    and so on.
    Thanks to the periodicity of $\omega^{2^mk}$, we do not have to compute the values for all $k$'s, but instead, compute eight of them (one for each $j$, twice) and re-use them in a clever fashion.
* **Number of ops:** $(N/2^m = 2) \times (2^m = 4) = 8= N$
    

#### Third step
* $m=1$, 
* $j=0, 1$,
* Compute the "partial" DFTs:
    \begin{align*}
        & E_0^{(1, j)} = E_0^{(2, j)} + \omega^{0\times2}E_0^{(2, j+2)}= E_0^{(2, j)} + E_0^{(2, j+2)},\\
        & E_1^{(1, j)} = E_1^{(2, j)} + \omega^{1\times2}E_1^{(2, j+2)} = E_1^{(2, j)} + \omega^2 E_1^{(2, j+4)},\\
    \end{align*}
    and we can use the trick where we re-use the $E_0$'s and $E_1$'s of the previous step to compute:
    \begin{align*}
        & E_2^{(1, j)} = E_0^{(2, j)} + \omega^{2\times2}E_0^{(2, j+2)}= E_0^{(2, j)} + \omega^4 E_0^{(2, j+4)},\\
        & E_3^{(1, j)} = E_1^{(2, j)} + \omega^{3\times2}E_1^{(2, j+2)} = E_1^{(2, j)} + \omega^6 E_1^{(2, j+4)}.
    \end{align*}
    We had to compute for twice as many $k$ indices, but for half as many $j$ indices.
* **Number of ops:** $(N/2^m = 4) \times (2^m = 2) = N$

#### Last step: the coefficients
* $m = 0$,
* $j=0$,
* We can now compute the DFT coefficients:
    \begin{align*}
        & E_0^{(0, 0)} = c_0 = E_0^{(1, 0)} + \omega^{0\times1}E_0^{(1, 1)} = E_0^{(1, 0)} + E_0^{(1, 1)},\\
        & E_1^{(0, 0)} = c_1 = E_1^{(1, 0)} + \omega^{1\times1}E_1^{(1, 1)} = E_1^{(1, 0)} + \omega E_1^{(1, 0)},\\
        & \vdots\\
        & E_7^{(0, 0)} = c_7 = E_1^{(3, j)} + \omega^7 E_1^{(1, 1)}.
    \end{align*}
* **Number of ops:** $(N/2^m = 8)\times (2^m = 1) = N$.
    
For each step ($m=2, 1, 0$, because we said we weren't counting $m=3$), we had 8 operations. $8\times 3=24 = 8\log_2(8)$, as promised!